# Business Insights with SQL

In this project we will work with data from the [Chinook database](https://github.com/lerocha/chinook-database). The Chinook database contains information about a fictional digital music shop such as data about the artists, songs, playlists, music genres and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases.

Just as a curiosity, the database's name is inspired in another database, the Northwind database. According to its creator, the Chinook database is named after the winds in the interior West of North America, where the Canadian Prairies and Great Plains meet various mountain ranges. The Chinooks are most common over southern Alberta in Canada and that is why he thought it would be I good name for a database that intented to be an alternative to Northwind.

With all information that Chinook contains, there is an infinity of answers we could get out of this database. Therefore, the main goal of this project, is to write some queries to demonstrate some of the insights we can get from the data and how they could be applied to the real world.

For the sake of simplicity, we'll assume that the fictional digital music shop is also named Chinook.

![image](music.jpg)

[Image source ](https://www.freepik.com/search?dates=any&page=1&query=music&sort=popular)

# The setup

In [23]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

# Understanding the database

The Chinook database contains eleven tables and every table is connected to one another. The following schema diagram will help us to understand the tables in this database as well as their relatioships to each other:

<img align="left" src = "chinook-schema.svg" width="800" height="500"/>

In [24]:
%%sql
SELECT 
    name, type
    FROM sqlite_master
    WHERE type='table';

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Analyse the performace of sales support agents 

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We can now analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

To do so we will find out the total sales done by each employee. To calculate the efficiency, we will find out the agents experience and then divide their total sales with their experience.

In [25]:
%%sql

SELECT
    e.first_name || ' ' || e.last_name AS employer_name,
    ROUND(SUM(i.total),2) AS total_sales,
    (CAST(julianday('now') -julianday(e.hire_date) AS INTEGER)) / 30 AS experience_months,
    ROUND(ROUND(SUM(i.total),2) / ((CAST(julianday('now') -julianday(e.hire_date) as INTEGER)) / 30), 2) as efficiency
    FROM employee AS e
    INNER JOIN customer AS c ON c.support_rep_id = e.employee_id
    INNER JOIN invoice AS i ON i.customer_id = c.customer_id
    GROUP BY employer_name
    ORDER BY efficiency DESC;

 * sqlite:///chinook.db
Done.


employer_name,total_sales,experience_months,efficiency
Jane Peacock,1731.51,38,45.57
Steve Johnson,1393.92,31,44.97
Margaret Park,1584.0,37,42.81


From the above table we can easily find out the performace of the agents.

# Analyzing Sales by Country

Now we will analyse sales by country.

The next query returns data on purchases from different countries. We will be able to see the total amount of sales as well as the average amount of dollar per customer and the average order value.

Notice that countries with only one costumer is grouped as 'Other'.

In [53]:
%%sql

/* Table that counts the number of invoices and sums the total purchase made
by each customer */

WITH information AS
(
    SELECT 
        c.customer_id,
        COUNT(i.invoice_id) AS purchase,
        c.country,
        SUM(i.total) AS total
        FROM customer AS c
        INNER JOIN invoice AS i ON i.customer_id = c.customer_id
        GROUP BY c.customer_id
),

/* Table to classify the country as "other" when the country has only 
one customer */

country_classifier AS
(
    SELECT 
        country,
        SUM(total) AS country_total,
        SUM(purchase) AS country_purchase,
        COUNT(customer_id) AS num_of_customers,
        CASE
            WHEN COUNT(customer_id) = 1 THEN 'other'
            ELSE country
        END AS new_country
        FROM information
        GROUP BY country
        ORDER BY country_total DESC
)

/* Here we will select all the required columns. In order to keep the 'other'
row as the last row we use a CASE statement.*/

SELECT 
    country,
    total_customers,
    total_sales, 
    average_value_per_customer,
    average_order_per_customer
    FROM 
        (
        SELECT
            new_country AS country,
            SUM(num_of_customers) AS  total_customers,
            SUM(country_total) AS total_sales,
            SUM(country_total) / SUM(country_purchase) AS average_order_per_customer,
            SUM(country_total) / SUM(num_of_customers) AS average_value_per_customer,
            CASE
                WHEN new_country = "other" THEN 1
                ELSE 0
            END AS sort
            FROM country_classifier
            GROUP BY new_country
            ORDER BY sort ASC, total_sales DESC
        )

 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,average_value_per_customer,average_order_per_customer
USA,13,1040.49,80.03769230769231,7.942671755725191
Canada,8,535.59,66.94875,7.047236842105264
Brazil,5,427.67999999999995,85.53599999999999,7.011147540983606
France,5,389.07,77.814,7.7814
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24,136.62,9.108
United Kingdom,3,245.51999999999998,81.83999999999999,8.768571428571429
Portugal,2,185.13,92.565,6.3837931034482756
India,2,183.14999999999998,91.57499999999999,8.72142857142857
other,15,1094.9399999999998,72.996,7.4485714285714275


The sample size here is not good enough, but if we have more data this query can provide some useful insights. For instance,  we can visualize which countries are already an established market for the company and also have an idea of which countries have more potential to become a successful market. For example, a country  although do not present a large amount of total sales, based on the present high average values per costumer and per order can be considered as promising markets.

Such insights are helpful for the company to decide the promising markets and the bugdet that needs to be allocated for such markets.

# Constomer preferences

Here we will focus on the preference of the customers such as the most popular genres and artists. This can be also really helpful for the company to decide on which genres and artists should they allocate more budget.

Since the company's target markets are country based, it is sensible to filter our query also country wise. 

# Popular genres in USA

For our example, we will consider USA as our target market and identify the popular genres and the artists.

PS: Country information can be obtained from two tables customer and invoice. There could be instances where a customer can have one coutry in customer table and another as billing country in invoice table. To avoid this problem, in our example, we are fetching country information from customer data.

In [54]:
%%sql

/* Table that filters customers from USA */

WITH usa AS
    (
     SELECT invl.*
        FROM customer AS c
        INNER JOIN invoice AS i ON i.customer_id = c.customer_id
        INNER JOIN invoice_line AS invl ON invl.invoice_id = i.invoice_id
        WHERE c.country = 'USA'
    )
SELECT
    g.name AS genre_name,
    COUNT(*) AS absolute_count,
    CAST(COUNT(usa.invoice_id)AS FLOAT) / (SELECT COUNT(*) FROM usa) as percentage
    FROM usa
    INNER JOIN track AS t ON t.track_id = usa.track_id
    INNER JOIN genre AS g on g.genre_id = t.genre_id
    GROUP BY g.name
    ORDER BY absolute_count DESC
    LIMIT 10;

 * sqlite:///chinook.db
Done.


genre_name,absolute_count,percentage
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


# Most popular artists in USA

Next we will identify the most popular artists in USA.

In [55]:
%%sql

/* Table that filters customers from USA */

WITH usa AS
    (
     SELECT invl.*
        FROM customer AS c
        INNER JOIN invoice AS i ON i.customer_id = c.customer_id
        INNER JOIN invoice_line AS invl ON invl.invoice_id = i.invoice_id
        WHERE c.country = 'USA'
    )
SELECT
    at.name AS artist_name,
    COUNT(*) AS times_purchased
    FROM usa
    INNER JOIN track AS t ON t.track_id = usa.track_id
    INNER JOIN album AS a on a.album_id = t.album_id
    INNER JOIN artist AS at on at.artist_id = a.artist_id
    GROUP BY at.name
    ORDER BY times_purchased DESC
    LIMIT 10;

 * sqlite:///chinook.db
Done.


artist_name,times_purchased
Van Halen,43
R.E.M.,38
The Rolling Stones,37
Nirvana,35
Foo Fighters,34
Eric Clapton,34
Guns N' Roses,32
Green Day,32
Pearl Jam,31
Amy Winehouse,30


# Purchase type : Albums or Individual Tracks

Some customers buy all the tracks from an album whereas some buy only selected ones. This information can be useful for the company to analyse. Company can save money by buying only popular individaul tracks from the recorders rather than buying the complete album. This can possibly be a money saving option too which company can ponder about.

This can also be done based on country, but in our example query, we will be considering a general scenario.

In [29]:
%%sql

/* Table that counts number of tracks per invoice */

WITH invoice_tracks AS
(
    SELECT
        il.invoice_id, 
        a.album_id,
        COUNT(DISTINCT t.track_id) AS inv_tracks
        FROM invoice_line AS il
        INNER JOIN track AS t ON t.track_id = il.track_id
        INNER JOIN album AS a ON a.album_id = t.album_id
        GROUP BY il.invoice_id, a.album_id
),

/* Table that counts number of tracks per album */

album_tracks AS
(
    SELECT
        a.album_id,
        COUNT(DISTINCT t.track_id) AS alb_tracks
        FROM track AS t
        INNER JOIN album AS a ON a.album_id = t.album_id
        GROUP BY a.album_id
),

/* Table that split the invoice based on the tracks as 0 (not album) and 
1 (album). For eg. if an invoice has 12 tracks where 10 of them belongs to 
one album which also has a total of 10 tracks, and the remaining two tracks 
belongs to another album which has more tracks, then this invoice will be 
split into two, the 10 tracks will be considered as 1(album) and the other 
two tracks as 0(not album). */

category AS
(
    SELECT *,
        CASE
            WHEN at.alb_tracks = it.inv_tracks AND at.album_id = it.album_id
            THEN 1
            ELSE 0 
        END AS category
        FROM album_tracks AS at
        INNER JOIN invoice_tracks AS it ON it.album_id = at.album_id
),

/* Table that classify the invoice into 'Album' or 'Not Album' based on the 
above category table.If an invoice already has an album in that (means if 
its already assigned 1 in the category table above), then it will be cosidered
as 'Album' otherwise 'Not Album'. */

album_classifier AS
(
    SELECT 
        invoice_id,
        CASE 
            WHEN MAX(category) = 0 THEN 'Not Album' 
            ELSE 'Album' 
        END AS purchase_type
        FROM category
        GROUP BY invoice_id
)
/* this is comment */

SELECT 
    purchase_type,
    COUNT(invoice_id) AS total_invoice,
    100 * (CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(invoice_id) FROM album_classifier))
    AS percentage
    FROM album_classifier
    GROUP BY purchase_type;

 * sqlite:///chinook.db
Done.


purchase_type,total_invoice,percentage
Album,201,32.73615635179153
Not Album,413,67.26384364820846


As it is clear from the above result that the customers purchasing a complete album is quite less, it may be a cheaper option for the company to consider in purchasing the tracks separately from the recorders than buying the whole album. 

# Stock details

Now we got the idea of costomer preferences, we can find out how good is the company stocks matching against customer preferences.

We already identified the popular genres and the artists among the costomers (of course in the USA market). But let us identify how the company stocks matches these demands.

# Most common genres in the stock

In [46]:
%%sql

SELECT 
    g.name AS genre_name,
    COUNT(*) AS total
    FROM genre AS g 
    INNER JOIN track AS t ON t.genre_id = g.genre_id
    GROUP BY g.name
    ORDER BY total DESC
    LIMIT 10;

 * sqlite:///chinook.db
Done.


genre_name,total
Rock,1297
Latin,579
Metal,374
Alternative & Punk,332
Jazz,130
TV Shows,93
Blues,81
Classical,74
Drama,64
R&B/Soul,61


# Most popular artists in the stock

In [47]:
%%sql

SELECT 
    at.name AS artist_name,
    COUNT(*) AS total
    FROM artist AS at 
    INNER JOIN album AS a ON a.artist_id = at.artist_id
    INNER JOIN track AS t ON t.album_id = a.album_id
    GROUP BY at.name
    ORDER BY total DESC
    LIMIT 10;

 * sqlite:///chinook.db
Done.


artist_name,total
Iron Maiden,213
U2,135
Led Zeppelin,114
Metallica,112
Lost,92
Deep Purple,92
Pearl Jam,67
Lenny Kravitz,57
Various Artists,56
The Office,53


# Conclusion

The goal of this project was to answer some important business questions using SQL that would help the company in making necessary changes in their future investment plans. We were able to do that successfully and also get insight on some other topics as well. We were able to understand the customer preferences and how much the company stocks are matching against those demands. We were also able to identify the potential markets where company can focus. We also identified one area where company probably could save money that is by purchasing tracks separately from the recorders than the whole album.  

We can hand over our findings and analysis to the corresponding team who can further investigate before making final decisions.